In [ ]:
# Commission Distribution Analysis

This notebook analyzes commission data to correctly assign commissions
to agents, including cases where a lead agent receives commissions
on behalf of other agents.

## Objectives
- Reconcile agents and clients across datasets
- Identify clients missing from the master list
- Validate data integrity during merges
- Generate a final commission summary by agent

In [ ]:
import pandas as pd
import os

# =====================================================
# Load data
# =====================================================
df_limpia = pd.read_excel("limpia.xlsx")
df_comision = pd.read_excel("comisiones.xlsx")

special_agent = "Lead Agent"

# =====================================================
# Initial integrity checks
# =====================================================
control_clientes = df_comision["Member_ID"].count()
control_comision = df_comision["Monto"].sum()

# =====================================================
# Identify members in commissions not present in master list
# =====================================================
no_en_limpia = df_comision[
    ~df_comision["Member_ID"].isin(df_limpia["Member_ID"])
]

# =====================================================
# Focus on specific agent (manual review case)
# =====================================================
special_agent_df = no_en_limpia[
    no_en_limpia["Agente"] == special_agent
]

# =====================================================
# Verify and clean duplicates in master list
# =====================================================
df_limpia = df_limpia.drop_duplicates(
    subset="Member_ID", keep="first"
)

# =====================================================
# Identify new clients from commissions (excluding special agent)
# =====================================================
clientes_en_comisiones = (
    df_comision[["Member_ID", "Agente"]]
    .drop_duplicates()
)

clientes_nuevos = clientes_en_comisiones[
    ~clientes_en_comisiones["Member_ID"].isin(df_limpia["Member_ID"])
]

clientes_nuevos = clientes_nuevos[
    clientes_nuevos["Agente"] != special_agent
]

df_limpia = pd.concat(
    [df_limpia[["Member_ID", "Agente"]], clientes_nuevos],
    ignore_index=True
)

# =====================================================
# Handle special agent separately
# =====================================================
special_agent_clientes = (
    df_comision[df_comision["Agente"] == special_agent]
    [["Member_ID", "Agente"]]
    .drop_duplicates()
)

special_agent_nuevos = special_agent_clientes[
    ~special_agent_clientes["Member_ID"].isin(df_limpia["Member_ID"])
]

df_limpia = pd.concat(
    [df_limpia, special_agent_nuevos],
    ignore_index=True
)

# =====================================================
# Prepare clean key for merge
# =====================================================
df_limpia_key = (
    df_limpia[["Member_ID", "Agente"]]
    .rename(columns={"Agente": "Agente_Real"})
)

# =====================================================
# Merge commissions with real agent
# =====================================================
df_comision_merged = df_comision.merge(
    df_limpia_key,
    on="Member_ID",
    how="left",
    validate="many_to_one"
)

# =====================================================
# Final integrity checks
# =====================================================
assert df_comision.shape[0] == df_comision_merged.shape[0]
assert df_comision["Monto"].sum() == df_comision_merged["Monto"].sum()
assert df_comision_merged["Member_ID"].count() == control_clientes

# =====================================================
# Commission summary by agent
# =====================================================
resumen_agente = (
    df_comision_merged
    .groupby("Agente_Real", as_index=False)
    .agg({"Monto": "sum"})
)

# =====================================================
# Export output
# =====================================================
os.makedirs("outputs", exist_ok=True)

resumen_agente.to_excel(
    "outputs/commission_summary_by_agent.xlsx",
    index=False
)